# VISION PIPELINE

## Download/Load Data

In [ ]:
import fastai
print(f"FastAi: {fastai.__version__}")

# Import all modules and functions from Fastai's vision package
from fastai.vision.all import * 

# Import get_image_files function for retrieving image file paths
from fastai.data.transforms import get_image_files

# Download and untar the PETS dataset for data preparation
path = untar_data(URLs.PETS)

# Get a list of image file names in the 'images' directory
fnames = get_image_files(path=path/'images')

# Print the total number of images and display the first 10 file names
print(f"Total {len(fnames)} images, {fnames[:10]} ...")

## Data Preparation

In [ ]:
# Load statistics typically used for image data normalization
from fastai.vision.core import imagenet_stats
# Import image augmentation and cropping transformations
from fastai.vision.augment import aug_transforms, RandomResizedCrop

# Define item transforms for image resizing and cropping
item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1., 1.))

# Define batch transforms with data augmentation and normalization
batch_tfms = [
    *aug_transforms(size=224, max_warp=0),  # Augmentations
    Normalize.from_stats(*imagenet_stats)   # Data normalization
]

## DataLoader

In [ ]:
# Use Fastai's DataBlock to specify data processing steps
from fastai.vision.data import ImageBlock
from fastai.data.block import CategoryBlock, DataBlock

# Get image files, split dataset, and label categories using a regular expression
from fastai.data.transforms import get_image_files, Normalize, RandomSplitter, RegexLabeller

# Define a DataBlock 'pets' for image classification
pets = DataBlock(
    blocks=(ImageBlock, CategoryBlock),  # Image and category blocks
    get_items=get_image_files,           # Get image files from a directory
    splitter=RandomSplitter(),           # Randomly split data into train and valid sets
    get_y=RegexLabeller(pat=r'/([^/]+)_\d+.*'),  # Extract category labels from file names
    item_tfms=item_tfms,                 # Item-level transformations
    batch_tfms=batch_tfms                # Batch-level transformations
)

# Create data loaders (dls) using the 'pets' DataBlock
dls = pets.dataloaders(source=path/'images', bs=64)

# Visualize a batch of preprocessed images
dls.show_batch()

# Print the number of classes in the vocabulary
print(f"{len(dls.vocab)} {dls.vocab}")

## Define Model & Find Learning Rate

In [ ]:
# Enable mixed-precision training
from fastai.callback.fp16 import to_fp16

# Create a vision learner with a ResNet-50 architecture
from fastai.vision.all import vision_learner
from torchvision.models.resnet import resnet50

# Import accuracy and error_rate metrics
from fastai.metrics import accuracy, error_rate

# Create a learner for image classification
learn = vision_learner(
    dls=dls,                  # Data loaders for training and validation
    arch=resnet50,            # Architecture (ResNet-50)
    pretrained=True,          # Use a pre-trained model
    metrics=[accuracy, error_rate]  # List of metrics including accuracy
).to_fp16()                  # Enable mixed-precision training

# Find the optimal learning rate for training
learn.lr_find()

## Model Training

In [ ]:
# Learning rate slice from 1e-4 to 1e-2
lr = slice(1e-4, 1e-2)

# Train the model for 4 epochs with a learning rate within the defined range
learn.fit_one_cycle(4, lr_max=lr)

# Save Model
# Save the trained model with the name 'model1_freezed'
learn.save('model1_freezed')

## Model Interpretation

In [ ]:
# Model Interpretation
from fastai.interpret import ClassificationInterpretation

interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()

print(len(dls.valid_ds)==len(losses)==len(idxs)) # True

interp.plot_top_losses(k=9, figsize=(15,10)) # pred/actual/loss/prob

In [ ]:
interp.plot_confusion_matrix(figsize=(16,16), dpi=60)

## Unfreeze Model, Fine Tune, Learning Rate

In [ ]:
# Load a previously trained model ('model1_freezed')
learn.load('model1_freezed')

# Find the optimal learning rates for fine-tuning
learn.lr_find()

In [ ]:
# Define a learning rate range (slice) for fine-tuning
lr = slice(1e-5, 1e-4)

# Unfreeze the model to allow all layers to be trained
learn.unfreeze()

# Fine-tune the model for 4 epochs with a learning rate within the defined range
learn.fit_one_cycle(4, lr_max=lr)

# Save the fine-tuned model with the name 'model2_unfreezed'
learn.save('model2_unfreezed')

## Inference

In [ ]:
!wget -O pug.jpg "https://th.bing.com/th/id/OIP.t7e7M-C6RM8T9a5xMgBsYgHaE8?w=273&h=182&c=7&r=0&o=5&dpr=2&pid=1.7"

In [ ]:
!tree /kaggle

In [ ]:
# Load the trained model
learn.load('model2_unfreezed')

In [ ]:
from PIL import Image
# Prepare the input image (you should replace 'image_path' with the path to your image)
input_image = Image.open('/kaggle/working/pug.jpg')
input_image

In [ ]:
# Perform inference
prediction = learn.predict(input_image)

# Display the results
print("Predicted Class:", prediction[0])
print("Predicted Probabilities:", prediction[2])

In [ ]:
prediction

In [ ]:
dls.vocab[29]

In [ ]:
path